In [1]:
from pymongo import MongoClient
from gridfs import GridFS
import psycopg2

import pandas as pd
import re
import tqdm
import requests
import urllib.request as request
from bs4 import BeautifulSoup
import json

In [2]:
# 建立mongodb连接
# mongoClient = MongoClient('mongodb://localhost:27017/')
mongoClient = MongoClient('mongodb://172.10.10.48:27017/') 
dtmr_dev = mongoClient.get_database('dtmr_dev')
dtmr_dev.authenticate('dtmr_dev', 'dtmr_dev')
dev = dtmr_dev.get_collection('ProteinStructure')
gfs_pdb = GridFS(dtmr_dev, collection='PDB_files')
gfs_seq = GridFS(dtmr_dev, collection='protein_seq_files')

In [3]:
length = len(pd.read_csv("PDB_IDS_FOR_FILES.csv").keyword.values)

In [4]:
# records = []
code = dev.count_documents({})
print(code, "of", length, ",", round(code/length*100, 2), "% done")
pdb_ids = pd.read_csv("PDB_IDS_FOR_FILES.csv").keyword.values[code:]

for pdb_id in tqdm.tqdm(pdb_ids):
    record = {}
    html_url = "https://www.rcsb.org/structure/" + pdb_id
    response = request.urlopen(html_url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    
    # 编号，例如PROT000001
    code = code + 1
    record['code'] = 'PROT'+ '0' * (6-len(str(code))) + str(code)
    record['PDBID'] = pdb_id

    LigandsMainTable = soup.find("table", id="LigandsMainTable")
    if LigandsMainTable == None:
        record["ligands"] = None
    else:
        ligand_rows = LigandsMainTable.find_all('tr', {'id' : re.compile('ligand_row_*')})
        ligands = []
        for row in ligand_rows:
            col = row.find_all("td")[2]
            ligand = {}
            ligand["Name"] = col.find("strong").text
            ligand["InChI Key"] = str(col.decode_contents()).split("<br/>")[-1]
            ligands.append(ligand)
        record["ligands"] = ligands
    
    
    r = requests.get("https://www.rcsb.org/fasta/entry/" + pdb_id, allow_redirects=True)
    if r.status_code != 200:
        record["sequence"] = None
    else:
        record["sequence"] = gfs_seq.put(r.content, filename=pdb_id+".fasta", type="fasta", keyword=pdb_id)

    r = requests.get("https://files.rcsb.org/download/" + pdb_id + ".pdb", allow_redirects=True)
    if r.status_code != 200:
        record["structure3d"] = None
    else:
        record["structure3d"] = gfs_pdb.put(r.content, filename=pdb_id+".pdb", type="pdb", keyword=pdb_id)
    
#     records.append(record)
    result = dev.insert_one(record)

329 of 7656 , 4.297283176593521 % done


  3%|██▌                                                                          | 244/7327 [19:20<9:21:34,  4.76s/it]


HTTPError: HTTP Error 504: Gateway Time-out